In [1]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import geopandas as gpd
import numpy as np
import osmnx as ox
import pickle

from shapely.geometry import Point
from geopy.distance import geodesic
from networkx import connected_components
from shapely.geometry import box

import utils_RNVI as utils_RNVI


## Methods

In [2]:
def compute_average_travel_time(G, sources, targets, weight='time'):

    path_lengths = []

    for target_node in targets:

        # associated police first responders
        source_first_responders = sources[target_node]
        
        for source_node in source_first_responders:
            if target_node != source_node:
                try:
                    # compute the shortest path length between the source and police station
                    length = nx.shortest_path_length(G, source=source_node, target=target_node, weight=weight)
                    path_lengths.append(length)
                except nx.NetworkXNoPath:
                    continue

    if path_lengths:
        return sum(path_lengths) / len(path_lengths)
    else:
        print('No paths found')
        return None
    


def Compute_Districts_Travel_Time(G, districts_labels, sampled_targets, closest_first_responders):
        
    # baseline computation
    district_travel_time = {stataustri_code: None for stataustri_code in districts_labels}

    for stataustri_code in districts_labels:
            
        # sampled nodes in the district
        targets = sampled_targets[sampled_targets['STATAUSTRI']==stataustri_code]['osmid'].tolist()

        # associated police stations as source of the service
        sources_dict = {osmid: closest_first_responders[osmid] for osmid in targets}
        
        # compute travel time
        average_travel_time = compute_average_travel_time(G, 
                                                        sources=sources_dict, 
                                                        targets=targets, 
                                                        weight='time')
            
        district_travel_time[stataustri_code] = average_travel_time

    return district_travel_time



def compute_district_impact_index(
    gdf_edges,
    gdf_nodes,
    G_undirected,
    districts_labels,
    node_counts_by_district,
    DISTRICT_ATTACKED,
    X=3000,
    N_RESPONDERS=3,
    RESPONDER_TYPE = 'police',
    percentage=10,
):
    """
    compute the district impact index after perturbing the network by removing attacked edges and considering 
    the impact on travel times across districts between a set of targets areas and first responders nodes as sources.

    Parameters:
    - gdf_edges: GeoDataFrame of edges.
    - gdf_nodes: GeoDataFrame of nodes.
    - G_undirected: Original undirected graph.
    - districts_labels: list of district codes.
    - node_counts_by_district: Dictionary of node counts by district.
    - DISTRICT_ATTACKED: District to attack by removing edges.
    - X: Number of edges to remove.
    - RESPONDER_TYPE: node type of responder (police or hospitals)
    - N_RESPONDERS: Number of first responders stations to associate as sources of service for each target
    - percentage: Percentage of target nodes to sample randomly by district.

    Returns:
    - district_impact_index: Dictionary with district codes as keys and impact indices as values.
    """

    # init district impact index
    district_impact_index = {stataustri_code: None for stataustri_code in districts_labels}

    # edges to remove given total length X (approximate area ensuring connectedness)
    selected_edges_gdf = utils_RNVI.select_edges_random_connected(gdf_edges, X, DISTRICT_ATTACKED)
    G_perturbed = utils_RNVI.remove_edges(G_undirected, selected_edges_gdf)

    # keep gcc
    gcc_nodes = max(connected_components(G_perturbed), key=len)
    G_perturbed_GCC = G_perturbed.subgraph(gcc_nodes).copy()

    # remove nodes from gdf_nodes that are not part of the GCC
    gdf_nodes_GCC = gdf_nodes[gdf_nodes['osmid'].isin(gcc_nodes)]

    # sample targets
    sampled_targets = utils_RNVI.select_random_nodes_by_district(
        gdf_nodes_GCC, node_counts_by_district, percentage=percentage
    )

    # compute closest police stations before and after perturbation
    closest_FR_before = utils_RNVI.associate_closest_responders(
        gdf_nodes, sampled_targets, N_RESPONDERS = N_RESPONDERS, RESPONDER_TYPE = RESPONDER_TYPE
    )
    closest_FR_after = utils_RNVI.associate_closest_responders(
        gdf_nodes_GCC, sampled_targets, N_RESPONDERS = N_RESPONDERS, RESPONDER_TYPE = RESPONDER_TYPE
    )

    # compute baseline travel times
    baseline_travel_time = Compute_Districts_Travel_Time(
        G_undirected, districts_labels, sampled_targets, closest_FR_before
    )

    # compute perturbed travel times
    perturbed_travel_time = Compute_Districts_Travel_Time(
        G_perturbed, districts_labels, sampled_targets, closest_FR_after
    )

    # Step 8: Compute the district impact index
    for code in districts_labels:
        district_impact_index[code] = perturbed_travel_time[code] / baseline_travel_time[code]

    return district_impact_index


## load spatial and network data

In [3]:
shapefile_path = "../data/BEZIRKSGRENZEOGD/BEZIRKSGRENZEOGDPolygon.shp"
# source: https://www.data.gv.at/katalog/en/dataset/stadt-wien_bezirksgrenzenwien

shapefile_path_census = "../data/ZAEHLBEZIRKOGD/ZAEHLBEZIRKOGDPolygon.shp"

# Load the shapefile into a GeoDataFrame
gdf_districts = gpd.read_file(shapefile_path)
gdf_census = gpd.read_file(shapefile_path_census)

gdf_districts['DISTRICT_CODE'] = gdf_districts.apply( lambda row: str(int(row['STATAUSTRI']*100)), axis = 1)
gdf_districts = gdf_districts.to_crs(epsg=4326)

In [4]:
# geojson of gdf edges and nodes
gdf_nodes = gpd.read_file("../processed/wien_gdf_nodes.geojson")
gdf_edges = gpd.read_file("../processed/wien_gdf_edges.geojson")

# laod graph of road network
with open("../processed/wien_G_undirected_new.pkl", "rb") as f:
    #G_undirected_load = pickle.load(f)
    G_undirected = pickle.load(f)

In [5]:
for u, v, data in G_undirected.edges(data=True):
    if 'maxspeed' in data and 'length' in data:
        try:
            # check if 'maxspeed' is a list
            if isinstance(data['maxspeed'], list):
                print(f"Issue: 'maxspeed' is a list for edge ({u}, {v}). Value: {data['maxspeed']}")
                data['time'] = None
                continue

            maxspeed = float(data['maxspeed']) * (1000 / 3600)  # convert to m/s
            length = float(data['length'])
            
            if maxspeed > 0:
                data['time'] = length / maxspeed
            else:
                print(f"Issue: 'maxspeed' is non-positive for edge ({u}, {v}). Value: {data['maxspeed']}")
                data['time'] = None
        except (ValueError, TypeError):
            print(f"Issue: unable to convert 'maxspeed' to float for edge ({u}, {v}). Value: {data['maxspeed']}")
            data['time'] = None
    else:
        print(f"Issue: missing 'maxspeed' or 'length' for edge ({u}, {v}).")
        data['time'] = None

In [8]:
#gdf_nodes, gdf_edges = ox.graph_to_gdfs(G_undirected_load, nodes=True, edges=True)
gdf_nodes = gdf_nodes.reset_index()
gdf_edges = gdf_edges.reset_index()

if 1:
    gdf_edges['centroid'] = gdf_edges.geometry.centroid
    G_undirected = G_undirected.subgraph(gdf_nodes.osmid).copy()

/tmp/ipykernel_56734/1956869581.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_edges['centroid'] = gdf_edges.geometry.centroid


In [11]:
node_counts_by_district = utils_RNVI.get_node_counts_by_district(gdf_nodes)
length_by_district = utils_RNVI.calculate_cumulative_length_by_district(gdf_edges)

limited_districts = list(gdf_nodes.STATAUSTRI.explode().unique())
gdf_districts = gdf_districts[gdf_districts['STATAUSTRI'].isin(limited_districts)]

districts_labels = list(gdf_districts.STATAUSTRI)

## MAIN

Current parameters:

- N_RUNS: numbers of simulations of an attack in each district
- X values: sets of lengths in meters of connected roads attacked in each scenario (then mapped to an approximate area of attack, while conserving connectedness of roads)
- RESPONDER_TYPE: hospitals or police
- N_RESPONDERS: number of responders stations per target node

### other variables in the simulation schedule

- fraction of sources used (percentage)

In [ ]:
# parameters

N_RUNS = 15
X_values = [2000, 4000, 6000]
RESPONDER_TYPE = 'hospitals' 
N_RESPONDERS = 3

In [ ]:
# main RUN

for X in X_values:
    for RUN in range(1, N_RUNS + 1):

        district_impact_overall = {code: None for code in districts_labels}

        for DISTRICT_ATTACKED in districts_labels:
            
            district_impact_index = compute_district_impact_index(
                gdf_edges, gdf_nodes, G_undirected, districts_labels,
                node_counts_by_district,
                DISTRICT_ATTACKED,
                X=X,
                N_RESPONDERS=N_RESPONDERS,
                RESPONDER_TYPE = RESPONDER_TYPE,
                percentage=15)
            
            district_impact_overall[DISTRICT_ATTACKED] = district_impact_index

        # runs results as dictionary
        run_results = {
            "District_Attacked": [],
            "District_Impacted": [],
            "Impact_Value": []
        }

        for district_attacked, targets in district_impact_overall.items():
            for district_impacted, impact_value in targets.items():
                run_results["District_Attacked"].append(district_attacked)
                run_results["District_Impacted"].append(district_impacted)
                run_results["Impact_Value"].append(impact_value)

        df_run_results = pd.DataFrame(run_results)

        output_file = f"../processed/simulations_{RESPONDER_TYPE}/small_district_impact_results_size_{X}_run_{RUN}.csv"
        df_run_results.to_csv(output_file, index=False)

        print(f"run {RUN} for size {X} completed")

/home/sbontorin/precrisis-wien/Wien-VRI_v1_server/code/utils_tmp.py:20: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  fr_stations['distance'] = fr_stations['geometry'].distance(current_geometry)
/home/sbontorin/miniconda3/envs/py3/lib/python3.10/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/home/sbontorin/precrisis-wien/Wien-VRI_v1_server/code/utils_tmp.py:20: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projecte